In [ ]:
import os
from openvino.tools.ovc import convert_model
from openvino.runtime import serialize, Core

# 定义 ONNX 模型路径
onnx_model_path = r"F:\Datasets\umt\model\onnx\yolov8n_detection.onnx"

# 定义输出目录路径
output_dir = r"F:\Datasets\umt\model\openvino\yolov8n_detection"


if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory at: {output_dir}")
else:
    print(f"Output directory already exists at: {output_dir}")


try:

    ov_model = convert_model(input_model=onnx_model_path)  # 转换模型

    output_model_path = os.path.join(output_dir, "yolov8n_detection")
    serialize(ov_model, output_model_path + ".xml", output_model_path + ".bin")
    print(f"Model successfully converted and saved to: {output_dir}")

    core = Core()
    try:
        ov_model_loaded = core.read_model(output_model_path + ".xml")
        print("OpenVINO model loaded successfully. Model format is valid.")
    except Exception as load_error:
        print(f"Error loading OpenVINO model: {load_error}")
except Exception as e:
    print(f"Error during model conversion: {e}")

In [5]:
import os
import numpy as np
from openvino.tools.ovc import convert_model
from openvino.runtime import serialize, Core, PartialShape, Dimension, Type, Layout

def convert_to_openvino(onnx_path, output_dir, min_batch=1, max_batch=16):
    """
    ONNX模型转换为支持动态batch的OpenVINO模型
    
    参数:
        onnx_path (str): ONNX模型路径
        output_dir (str): 输出目录
        min_batch (int): 最小batch大小
        max_batch (int): 最大batch大小
    """
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n{'='*50}")
    print(f"Converting ONNX model to OpenVINO with dynamic batch [{min_batch}..{max_batch}]")
    print(f"ONNX model: {onnx_path}")
    print(f"Output dir: {output_dir}")
    print(f"{'='*50}\n")

    try:
        # 步骤1: 转换模型
        print("Step 1: Converting model...")
        ov_model = convert_model(
            input_model=onnx_path
        )

        # 步骤2: 显式设置动态维度
        print("\nStep 2: Setting dynamic dimensions...")
        for input_layer in ov_model.inputs:
            input_name = input_layer.any_name
            input_layer.get_node().set_partial_shape(
                PartialShape([
                    Dimension(min_batch, max_batch),  # 动态batch
                    Dimension(3),                     # 通道
                    Dimension(640),                   # 高度
                    Dimension(640)                    # 宽度
                ])
            )
            input_layer.get_node().set_element_type(Type.f32)
            input_layer.get_node().set_layout(Layout("NCHW"))
            print(f"  Set dynamic shape for {input_name}: [{min_batch}..{max_batch},3,640,640]")

        # 步骤3: 保存模型
        print("\nStep 3: Saving model...")
        model_name = os.path.splitext(os.path.basename(onnx_path))[0]
        output_path = os.path.join(output_dir, model_name)
        serialize(ov_model, f"{output_path}.xml", f"{output_path}.bin")
        print(f"  Model saved to: {output_path}.xml/.bin")

        # 步骤4: 验证模型
        print("\nStep 4: Validating model...")
        core = Core()
        model = core.read_model(f"{output_path}.xml")
        
        print("\nModel inputs (PartialShape):")
        for idx, input in enumerate(model.inputs):
            print(f"  Input {idx}: {input.any_name} {input.partial_shape} {input.element_type}")
        
        compiled_model = core.compile_model(model, "AUTO")
        input_name = compiled_model.input(0).any_name
        
        # 测试不同batch大小
        print("\nTesting dynamic batch:")
        test_batches = [min_batch, max_batch, (min_batch + max_batch)//2]
        
        for batch_size in test_batches:
            try:
                input_shape = [batch_size, 3, 640, 640]
                dummy_input = {input_name: np.random.randn(*input_shape).astype(np.float32)}
                
                results = compiled_model(dummy_input)
                print(f"  Successfully inferred with batch={batch_size}")
                
                for output in compiled_model.outputs:
                    print(f"    Output '{output.any_name}' shape: {results[output]}")
                    
            except Exception as e:
                print(f"  Error with batch {batch_size}: {str(e)}")
                raise

        print("\nConversion and validation completed successfully!")
        return True

    except Exception as e:
        print(f"\nError during conversion: {str(e)}")
        return False

if __name__ == "__main__":

    ONNX_MODEL_PATH = r"F:\Studio\inferx\models\onnx\rtdetr-l.onnx"
    OUTPUT_DIR = r"F:\Studio\inferx\models\openvino"
    MIN_BATCH = 1  # 最小batch大小
    MAX_BATCH = 16  # 最大batch大小

    # 执行转换
    success = convert_to_openvino(
        onnx_path=ONNX_MODEL_PATH,
        output_dir=OUTPUT_DIR,
        min_batch=MIN_BATCH,
        max_batch=MAX_BATCH
    )

    if success:
        print("\nOpenVINO model with dynamic batch support is ready for deployment!")
    else:
        print("\nModel conversion failed. Please check the error messages.")


Converting ONNX model to OpenVINO with dynamic batch [1..16]
ONNX model: F:\Studio\inferx\models\onnx\rtdetr-l.onnx
Output dir: F:\Studio\inferx\models\openvino

Step 1: Converting model...

Step 2: Setting dynamic dimensions...
  Set dynamic shape for images: [1..16,3,640,640]

Step 3: Saving model...
  Model saved to: F:\Studio\inferx\models\openvino\rtdetr-l.xml/.bin

Step 4: Validating model...

Model inputs (PartialShape):
  Input 0: images [1..16,3,640,640] <Type: 'float32'>

Testing dynamic batch:
  Successfully inferred with batch=1
    Output 'output0' shape: [[[0.5004864  0.5008794  0.99998975 ... 0.03318341 0.01502175 0.06388879]
  [0.49915323 0.50246596 0.99999    ... 0.02416435 0.02616708 0.03792572]
  [0.49790093 0.5044255  0.9999874  ... 0.0011255  0.0010709  0.00120133]
  ...
  [0.5050216  0.3646722  0.99637014 ... 0.00151515 0.00200372 0.00380631]
  [0.47006902 0.70329434 0.02461881 ... 0.01005173 0.00316641 0.00502892]
  [0.18795946 0.01203666 0.0266506  ... 0.008975